<h2 style="font-weight: bold">Meets Bags of Popcorn</h2>

<h4>This is my first published notebook on Kaggle About NLP, So I decided ofc to take a look at the Bag of Words Meets Bags of Popcorn Compeition 😄😄<br><br>I will be doing a EDA of review texts, some Visualization and Pre-Processing. and finally modelling <br></h4>

* <h5 style="font-weight: 700">Your feedback is very welcome</h5>
* <h5 style="font-weight: 700">If you find this notebook useful, please don't forget to upvote it!</h5>

In [ ]:
import numpy as np
import pandas as pd
import re
import json
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from wordcloud import WordCloud
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
import zipfile

files=['../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip',
       '../input/word2vec-nlp-tutorial/testData.tsv.zip',
       '../input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip']

for file in files :
    zip = zipfile.ZipFile(file,'r')
    zip.extractall()
    zip.close()

In [ ]:
train=pd.read_csv('/kaggle/working/labeledTrainData.tsv', delimiter="\t")
test=pd.read_csv('/kaggle/working/testData.tsv', delimiter="\t")

In [ ]:
sub=pd.read_csv('/kaggle/input/word2vec-nlp-tutorial/sampleSubmission.csv')

In [ ]:
train.head()